In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [2]:
# 1. Data Preparation
def load_ipinyou_data(file_path):
    # Assuming CSV format for simplicity
    data = pd.read_csv(file_path)
    
    # Example preprocessing: converting categorical features to numerical
    label_encoders = {}
    for column in data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le
    
    # Splitting data into features (X) and label (y)
    X = data.drop(columns=['click'])  # Assuming 'click' is the target column
    y = data['click']
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train.values, X_test.values, y_train.values, y_test.values, label_encoders


In [3]:
# 2. Defining Models
class DeepFM(nn.Module):
    def __init__(self, feature_nums, field_nums, latent_dims):
        super(DeepFM, self).__init__()
        self.feature_embedding = nn.Embedding(feature_nums, latent_dims)
        nn.init.xavier_uniform_(self.feature_embedding.weight.data)
        
        # FM part
        self.fm_first_order = nn.Embedding(feature_nums, 1)
        self.fm_second_order = nn.Embedding(feature_nums, latent_dims)
        
        # Deep part
        self.deep = nn.Sequential(
            nn.Linear(field_nums * latent_dims, 400),
            nn.ReLU(),
            nn.Linear(400, 400),
            nn.ReLU(),
            nn.Linear(400, 1)
        )
    
    def forward(self, x):
        emb = self.feature_embedding(x)
        square_sum = torch.sum(emb, dim=1) ** 2
        sum_square = torch.sum(emb ** 2, dim=1)
        fm_part = 0.5 * (square_sum - sum_square)
        
        # First-order interactions
        first_order = torch.sum(self.fm_first_order(x), dim=1)
        
        # Deep part
        deep_part = self.deep(emb.view(emb.size(0), -1))
        
        return torch.sigmoid(first_order + fm_part + deep_part)

class FNN(nn.Module):
    def __init__(self, feature_nums, field_nums, latent_dims):
        super(FNN, self).__init__()
        self.embedding = nn.Embedding(feature_nums, latent_dims)
        self.fc = nn.Sequential(
            nn.Linear(field_nums * latent_dims, 400),
            nn.ReLU(),
            nn.Linear(400, 400),
            nn.ReLU(),
            nn.Linear(400, 1)
        )
    
    def forward(self, x):
        emb = self.embedding(x).view(x.size(0), -1)
        return torch.sigmoid(self.fc(emb))

class DCN(nn.Module):
    def __init__(self, feature_nums, field_nums, latent_dims):
        super(DCN, self).__init__()
        self.embedding = nn.Embedding(feature_nums, latent_dims)
        
        # Cross network part
        self.cross_weight = nn.Parameter(torch.randn(field_nums * latent_dims, 1))
        self.cross_bias = nn.Parameter(torch.zeros(1))
        
        # Deep part
        self.deep = nn.Sequential(
            nn.Linear(field_nums * latent_dims, 400),
            nn.ReLU(),
            nn.Linear(400, 400),
            nn.ReLU(),
            nn.Linear(400, 1)
        )
    
    def forward(self, x):
        emb = self.embedding(x).view(x.size(0), -1)
        
        # Cross Network
        cross_part = emb
        for _ in range(3):  # 3 layers of cross network
            cross_part = torch.matmul(emb, self.cross_weight) * cross_part + emb
        
        # Deep part
        deep_part = self.deep(emb)
        
        return torch.sigmoid(cross_part + deep_part)

class AFM(nn.Module):
    def __init__(self, feature_nums, field_nums, latent_dims):
        super(AFM, self).__init__()
        self.feature_embedding = nn.Embedding(feature_nums, latent_dims)
        
        # Attention mechanism for AFM
        self.attention_net = nn.Sequential(
            nn.Linear(latent_dims, latent_dims),
            nn.ReLU(),
            nn.Linear(latent_dims, 1)
        )
    
    def forward(self, x):
        emb = self.feature_embedding(x)
        
        # Pairwise interactions
        row, col = [], []
        for i in range(emb.size(1)):
            for j in range(i + 1, emb.size(1)):
                row.append(i)
                col.append(j)
        
        interactions = emb[:, row] * emb[:, col]
        
        # Attention weights
        attn_scores = self.attention_net(interactions)
        attn_output = torch.sum(attn_scores * interactions, dim=1)
        
        return torch.sigmoid(attn_output)

In [4]:
# 3. Training the Models
def train_model(model, X_train, y_train, X_test, y_test, config):
    model.to(config['device'])
    optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    criterion = nn.BCELoss()
    
    X_train = torch.LongTensor(X_train).to(config['device'])
    y_train = torch.FloatTensor(y_train).to(config['device'])
    X_test = torch.LongTensor(X_test).to(config['device'])
    y_test = torch.FloatTensor(y_test).to(config['device'])
    
    for epoch in range(config['epochs']):
        model.train()
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs.squeeze(), y_train)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            model.eval()
            test_outputs = model(X_test).detach()
            test_loss = criterion(test_outputs.squeeze(), y_test)
            print(f"Epoch [{epoch+1}/{config['epochs']}], Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}")


In [5]:
# 4. Configuration
config = {
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'lr': 0.001,
    'epochs': 50,
    'latent_dims': 10,
    'feature_nums': 1000,  # Adjust based on actual data
    'field_nums': 20  # Adjust based on actual data
}

In [ ]:
# 5. Execution
if __name__ == "__main__":
    # Example file path for iPinYou dataset
    file_path = "ipinyou.csv"
    
    # Load the data
    X_train, X_test, y_train, y_test, label_encoders = load_ipinyou_data(file_path)
    
    # Train DeepFM
    print("Training DeepFM...")
    deepfm_model = DeepFM(config['feature_nums'], config['field_nums'], config['latent_dims'])
    train_model(deepfm_model, X_train, y_train, X_test, y_test, config)
    
    # Train FNN
    print("Training FNN...")
    fnn_model = FNN(config['feature_nums'], config['field_nums'], config['latent_dims'])
    train_model(fnn_model, X_train, y_train, X_test, y_test, config)
    
    # Train DCN
    print("Training DCN...")
    dcn_model = DCN(config['feature_nums'], config['field_nums'], config['latent_dims'])
    train_model(dcn_model, X_train, y_train, X_test, y_test, config)
    
    # Train AFM
    print("Training AFM...")
    afm_model = AFM(config['feature_nums'], config['field_nums'], config['latent_dims'])
    train_model(afm_model, X_train, y_train, X_test, y_test, config)